In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=9c3c9f97126c16b40070c57f9ea983d6d03386a3f78bf469690823b2e8755d5c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
from __future__ import print_function
from scipy.io import loadmat
from tqdm import tqdm
from google.colab import drive
import numpy as np
import scipy
import os
import matplotlib.pyplot as plt
import csv
import pandas as pd
import wget
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [30]:
wget.download('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv', os.getcwd()+"/Global_Mobility_Report.csv")
g_list = pd.read_csv('Global_Mobility_Report.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
def normalize(arr):
  slope = int(arr[-1])-int(arr[-2])
  return int(arr[-1])+slope

In [17]:
def fn(mon):
  if(mon == "01"):
    return " January"
  if(mon == "02"):
    return " February"
  if(mon == "03"):
    return " March"
  if(mon == "04"):
    return " April"
  if(mon == "05"):
    return " May"
  if(mon == "06"):
    return " June"
  if(mon == "07"):
    return " July"
  if(mon == "08"):
    return " August"
  if(mon == "09"):
    return " September"
  if(mon == "10"):
    return " October"
  if(mon == "11"):
    return " November"
  if(mon == "12"):
    return " December"
def convert(dat): 
    return  str(dat[-2:]) + fn(str(dat[5:7]))

In [18]:
def isNaN(num):
    return num != num

In [28]:
# g_listNew = g_list[g_list['country_region'] == 'India']
# g_listNew = g_listNew.drop_duplicates(subset=['sub_region_1', 'date'])
# g_listNew

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
1009128,IN,India,NaN,NaN,NaN,NaN,NaN,2020-02-15,1.0,2.0,3.0,3.0,5.0,0.0
1009129,IN,India,NaN,NaN,NaN,NaN,NaN,2020-02-16,2.0,2.0,3.0,2.0,0.0,0.0
1009130,IN,India,NaN,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,3.0,1.0,4.0,0.0
1009131,IN,India,NaN,NaN,NaN,NaN,NaN,2020-02-18,0.0,2.0,4.0,2.0,3.0,0.0
1009132,IN,India,NaN,NaN,NaN,NaN,NaN,2020-02-19,0.0,2.0,1.0,1.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144607,IN,India,West Bengal,NaN,NaN,IN-WB,NaN,2020-09-07,-75.0,-57.0,-53.0,-74.0,-65.0,27.0
1144608,IN,India,West Bengal,NaN,NaN,IN-WB,NaN,2020-09-08,-30.0,15.0,-31.0,-32.0,-30.0,10.0
1144609,IN,India,West Bengal,NaN,NaN,IN-WB,NaN,2020-09-09,-29.0,18.0,-32.0,-30.0,-26.0,9.0
1144610,IN,India,West Bengal,NaN,NaN,IN-WB,NaN,2020-09-10,-23.0,28.0,-30.0,-28.0,-30.0,10.0


In [31]:
india_dict = {}
df=pd.DataFrame()
csv_state = []
csv_dates = []
csv_retail = []
csv_grocery = []
csv_parks = []
csv_transit = []
csv_workplace = []
csv_residential = []
csv_average_mobility = []
temp=0
columnDict = {}
g_list = g_list[g_list['country_region'] == 'India']
g_list = g_list.drop_duplicates(subset=['sub_region_1', 'date'])
for index, row in g_list.iterrows():
    if isNaN(row['sub_region_1']):
      row['sub_region_1']='India'
    if row['sub_region_1'] not in  india_dict.keys():
      india_dict[row['sub_region_1']] = {}
    state = row['sub_region_1']
    dates = row['date']
    # print(state," ",dates)
    retail = row['retail_and_recreation_percent_change_from_baseline']
    grocery = row['grocery_and_pharmacy_percent_change_from_baseline']
    parks = row['parks_percent_change_from_baseline']
    transit = row['transit_stations_percent_change_from_baseline']
    workplace = row['workplaces_percent_change_from_baseline']
    residential = row['residential_percent_change_from_baseline']
    if 'dates' not in india_dict[state].keys():
      india_dict[state]['dates'] = []
      india_dict[state]['retail'] = []
      india_dict[state]['grocery'] = []
      india_dict[state]['parks'] = []
      india_dict[state]['transit'] = []
      india_dict[state]['workplace'] = []
      india_dict[state]['residential'] = []
      india_dict[state]['average_mobility'] = []
    l = len(india_dict[state]['dates'])
    if(l>1 and l%7<2):
      india_dict[state]['workplace'].append(india_dict[state]['workplace'][-1])
      india_dict[state]['retail'].append(india_dict[state]['retail'][-1])
      india_dict[state]['grocery'].append(india_dict[state]['grocery'][-1])
      india_dict[state]['parks'].append(india_dict[state]['parks'][-1])
      india_dict[state]['transit'].append(india_dict[state]['transit'][-1])
      india_dict[state]['residential'].append(india_dict[state]['residential'][-1])
    else:
      india_dict[state]['workplace'].append(row['workplaces_percent_change_from_baseline'])
      india_dict[state]['retail'].append(row['retail_and_recreation_percent_change_from_baseline'])
      india_dict[state]['grocery'].append(row['grocery_and_pharmacy_percent_change_from_baseline'])
      india_dict[state]['parks'].append(row['parks_percent_change_from_baseline'])
      india_dict[state]['transit'].append(row['transit_stations_percent_change_from_baseline'])
      india_dict[state]['residential'].append(row['residential_percent_change_from_baseline'])
    temp=[]
    if(not isNaN((india_dict[state]['retail'][-1]))):
      temp.append(int(india_dict[state]['retail'][-1]))
    if(not isNaN((india_dict[state]['grocery'][-1]))):
      temp.append(int(india_dict[state]['grocery'][-1]))
    if(not isNaN((india_dict[state]['transit'][-1]))):
      temp.append(int(india_dict[state]['transit'][-1]))
    if(not isNaN((india_dict[state]['workplace'][-1]))):
      temp.append(int(india_dict[state]['workplace'][-1]))
    count = 0
    sum=0
    for i in temp:
      sum+=i
      count+=1
    if(len(temp)>0):
        avg=sum/count
        india_dict[state]['average_mobility'].append(str(avg))
    else:
        india_dict[state]['average_mobility'].append('')
#     if(len(india_dict[state]['retail'][-1])>0 and len(india_dict[state]['grocery'][-1])>0 and len(india_dict[state]['transit'][-1])>0 and len(india_dict[state]['workplace'][-1])>0 ):
#       avg = (int(india_dict[state]['retail'][-1])+int(india_dict[state]['grocery'][-1])+int(india_dict[state]['transit'][-1])+int(india_dict[state]['workplace'][-1]))/4
#       india_dict[state]['average_mobility'].append(str(avg))  # To change for average
#     else:
#       india_dict[state]['average_mobility'].append('')  # To change for average
    india_dict[state]['dates'].append(convert(row['date']))
      
    for i in range(len(india_dict[state]['dates'])):
      if (temp!=india_dict[state]['dates'][-1]) :
        csv_state.append(state)
        csv_dates.append(india_dict[state]['dates'][-1])
        csv_retail.append(india_dict[state]['retail'][-1])
        csv_grocery.append(india_dict[state]['grocery'][-1])
        csv_parks.append(india_dict[state]['parks'][-1])
        csv_transit.append(india_dict[state]['transit'][-1])
        csv_workplace.append(india_dict[state]['workplace'][-1])
        csv_residential.append(india_dict[state]['residential'][-1])
        csv_average_mobility.append(india_dict[state]['average_mobility'][-1])
        temp=india_dict[state]['dates'][-1]

Streaming output truncated to the last 5000 lines.
Haryana   2020-03-27
Haryana   2020-03-28
Haryana   2020-03-29
Haryana   2020-03-30
Haryana   2020-03-31
Haryana   2020-04-01
Haryana   2020-04-02
Haryana   2020-04-03
Haryana   2020-04-04
Haryana   2020-04-05
Haryana   2020-04-06
Haryana   2020-04-07
Haryana   2020-04-08
Haryana   2020-04-09
Haryana   2020-04-10
Haryana   2020-04-11
Haryana   2020-04-12
Haryana   2020-04-13
Haryana   2020-04-14
Haryana   2020-04-15
Haryana   2020-04-16
Haryana   2020-04-17
Haryana   2020-04-18
Haryana   2020-04-19
Haryana   2020-04-20
Haryana   2020-04-21
Haryana   2020-04-22
Haryana   2020-04-23
Haryana   2020-04-24
Haryana   2020-04-25
Haryana   2020-04-26
Haryana   2020-04-27
Haryana   2020-04-28
Haryana   2020-04-29
Haryana   2020-04-30
Haryana   2020-05-01
Haryana   2020-05-02
Haryana   2020-05-03
Haryana   2020-05-04
Haryana   2020-05-05
Haryana   2020-05-06
Haryana   2020-05-07
Haryana   2020-05-08
Haryana   2020-05-09
Haryana   2020-05-10
Hary

In [32]:
df=pd.DataFrame()
df['state']=csv_state
df['dates']=csv_dates
df['retail']=csv_retail
df['grocery']=csv_grocery
df['parks']=csv_parks
df['transit']=csv_transit
df['workplace']=csv_workplace
df['residential']=csv_residential
df['average_mobility']=csv_average_mobility
df.fillna(method='ffill')
df.to_csv('mobility.csv',index=False)

In [33]:
df

,state,dates,retail,grocery,parks,transit,workplace,residential,average_mobility
0,India,15 February,1.0,2.0,3.0,3.0,5.0,0.0,2.75
1,India,16 February,2.0,2.0,3.0,2.0,0.0,0.0,1.5
2,India,17 February,-1.0,1.0,3.0,1.0,4.0,0.0,1.25
3,India,18 February,0.0,2.0,4.0,2.0,3.0,0.0,1.75
4,India,19 February,0.0,2.0,1.0,1.0,4.0,1.0,1.75
...,...,...,...,...,...,...,...,...,...
7765,West Bengal,07 September,-75.0,-57.0,-53.0,-74.0,-65.0,27.0,-67.75
7766,West Bengal,08 September,-30.0,15.0,-31.0,-32.0,-30.0,10.0,-19.25
7767,West Bengal,09 September,-29.0,18.0,-32.0,-30.0,-26.0,9.0,-16.75
7768,West Bengal,10 September,-23.0,28.0,-30.0,-28.0,-30.0,10.0,-13.25


In [34]:
from datetime import datetime
import json
india_dict['datetime']=str(datetime.now())
mobility_json_indented = json.dumps(india_dict, indent = 4)
mobility_json = json.dumps(india_dict)


In [35]:
with open("india_mobility.json", "w") as outfile: 
    outfile.write(mobility_json)

In [36]:
with open("india_mobility_indented.json", "w") as outfile: 
    outfile.write(mobility_json_indented)